# Linear Regression Consulting Project

## Project objective:
Predict how many crew members a ship will need

## Project detail:

Hyundai Heavy Industries wants to build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

#### hint: use StringIndexer or OneHotEncoder

StringIndexer (equivalent to LabelEncoder in sklearn) assigns index [0-num_labels] to each string.
The problem here is, since there are different numbers in the same column, the model will misunderstand the data to be in some kind of order. But this is not the case at all. To overcome this problem, we use One Hot Encoder.


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MLlibLRProject').getOrCreate()


In [2]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('cruise_ship_info.csv',header=True, inferSchema=True)

In [6]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)

